In [11]:
#Import the necessary libraries
import os
import cv2
import numpy as np
import time

In [12]:
#Import Directory for OpenCV screenshots
screenshot_dir = 'CV_Screenshots'
os.makedirs(screenshot_dir, exist_ok=True)

In [13]:
#PHASE 1: CAMERA ACCESS AND SETUP

def find_working_camera():
    print("🔍 Searching for working camera...\n")
    
    # Try with AVFoundation (Mac-specific)
    cap = cv2.VideoCapture(1, cv2.CAP_AVFOUNDATION)
        
    if cap.isOpened():
            # Try to read a frame
        ret, frame = cap.read()
        if ret and frame is not None:
            h, w = frame.shape[:2]
            print(f"✅ WORKS! ({w}x{h})")
            cap.release()
            return 1
        else:
            print("❌ Opens but can't grab frames")
    else:
        print("❌ Can't open")
        
    cap.release()
    
    return None

In [14]:

def test_camera_access():
    # Find working camera
    camera_index = find_working_camera()
    
    if camera_index is None:
        print("\n❌ ERROR: No working camera found!")
        print("💡 Troubleshooting:")
        print("   1. Close any apps using camera (Zoom, Teams, FaceTime)")
        print("   2. System Settings > Privacy & Security > Camera")
        print("      → Enable for Terminal/Python/VS Code")
        print("   3. Restart your Python kernel/terminal")
        return
    
    print(f"\n✅ Using camera {camera_index}")
    
    # Open camera with AVFoundation backend (better for Mac)
    cap = cv2.VideoCapture(camera_index, cv2.CAP_AVFOUNDATION)
    
    # Force better settings
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    cap.set(cv2.CAP_PROP_FPS, 30)
    
    print("✅ Camera opened successfully!")
    print("\n📋 Instructions:")
    print("   - Press 'q' to quit")
    print("   - Press 's' to save screenshot")
    print("   - ESC also quits")
    print("\nCamera window should open now...\n")
    
    # Get camera properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    print(f"📊 Camera Info:")
    print(f"   Resolution: {width}x{height}")
    print(f"   FPS: {fps}")
    
    screenshot_count = 0
    frame_count = 0
    start_time = time.time()
    
    # Main camera loop
    while True:
        # Read frame from camera
        ret, frame = cap.read()
        
        if not ret:
            print("❌ Failed to grab frame")
            break
        
        frame_count += 1
        
        # Calculate actual FPS
        elapsed_time = time.time() - start_time
        if elapsed_time > 0:
            actual_fps = frame_count / elapsed_time
        else:
            actual_fps = 0
        
        # Add FPS counter to frame
        cv2.putText(
            frame, 
            f"FPS: {actual_fps:.1f}", 
            (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 
            0.7, 
            (0, 255, 0), 
            2
        )
        
        # Add instructions
        cv2.putText(
            frame, 
            "Press 'q' to quit, 's' for screenshot", 
            (10, height - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 
            0.5, 
            (255, 255, 255), 
            1
        )
        
        # Display the frame
        cv2.imshow('Friend Classifier - Phase 1', frame)
        
        # Handle keyboard input
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q') or key == 27:  # 'q' or ESC
            print("\n👋 Quitting...")
            break
        elif key == ord('s'):  # Save screenshot
            screenshot_count += 1
            filename = os.path.join(screenshot_dir, f'screenshot_{screenshot_count}.jpg')
            cv2.imwrite(filename, frame)
            print(f"📸 Screenshot saved: {filename}")
    
    # Cleanup
    cap.release()
    cv2.destroyAllWindows()
    
    print("\n" + "="*60)
    print("✅ PHASE 1 COMPLETE!")
    print(f"   Total frames: {frame_count}")
    print(f"   Average FPS: {actual_fps:.1f}")
    print(f"   Screenshots: {screenshot_count}")
    print("="*60)


In [15]:
# PHASE 2: FACE DETECTION

def load_face_detector():
    print("📦 Loading Haar Cascade face detector...")
    
    # Load pre-trained face detection model
    cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    face_cascade = cv2.CascadeClassifier(cascade_path)
    
    if face_cascade.empty():
        print("❌ ERROR: Could not load face detector!")
        print(f"   Tried path: {cascade_path}")
        return None
    
    print("✅ Face detector loaded successfully!")
    return face_cascade

In [16]:

def detect_faces(frame, face_cascade, scale_factor=1.1, min_neighbors=5, min_size=(30, 30)):
    # Convert to grayscale (Haar Cascades work on grayscale)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=scale_factor,
        minNeighbors=min_neighbors,
        minSize=min_size,
        flags=cv2.CASCADE_SCALE_IMAGE
    )
    
    return faces


In [17]:

def draw_face_boxes(frame, faces, color=(0, 255, 0), thickness=2):
    
    for (x, y, w, h) in faces:
        # Draw rectangle around face
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, thickness)
        
        # Add label above face
        label = "Face"
        label_y = y - 10 if y - 10 > 10 else y + h + 20
        
        cv2.putText(
            frame,
            label,
            (x, label_y),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            color,
            2
        )
    
    return frame

In [18]:

def face_detection_camera(camera_index=1):

    print("\n" + "="*60)
    print("🎥 PHASE 2: FACE DETECTION")
    print("="*60)
    
    # Load face detector
    face_cascade = load_face_detector()
    if face_cascade is None:
        return
    
    # Open camera (using your working camera index)
    print(f"\n📹 Opening camera {camera_index}...")
    cap = cv2.VideoCapture(camera_index, cv2.CAP_AVFOUNDATION)
    
    if not cap.isOpened():
        print("❌ Failed to open camera!")
        return
    
    # Set camera properties
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    
    print("✅ Camera opened successfully!")
    print("\n📋 Controls:")
    print("   q - Quit")
    print("   s - Save screenshot")
    print("   + - Increase sensitivity (detect more faces)")
    print("   - - Decrease sensitivity (detect fewer faces)")
    print("\nWindow opening...\n")
    
    # Detection parameters (adjustable)
    scale_factor = 1.1
    min_neighbors = 5
    
    # Statistics
    frame_count = 0
    faces_detected_total = 0
    start_time = time.time()
    screenshot_count = 0
    
    # Main loop
    while True:
        ret, frame = cap.read()
        
        if not ret:
            print("❌ Failed to grab frame")
            break
        
        frame_count += 1
        
        # Detect faces
        faces = detect_faces(
            frame, 
            face_cascade,
            scale_factor=scale_factor,
            min_neighbors=min_neighbors
        )
        
        faces_detected_total += len(faces)
        
        # Draw face boxes
        frame = draw_face_boxes(frame, faces)
        
        # Calculate FPS
        elapsed_time = time.time() - start_time
        fps = frame_count / elapsed_time if elapsed_time > 0 else 0
        
        # Add overlay information
        height, width = frame.shape[:2]
        
        # FPS counter
        cv2.putText(
            frame,
            f"FPS: {fps:.1f}",
            (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            (0, 255, 0),
            2
        )
        
        # Face count
        face_text = f"Faces: {len(faces)}"
        cv2.putText(
            frame,
            face_text,
            (10, 60),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            (0, 255, 0),
            2
        )
        
        # Detection parameters
        param_text = f"Scale: {scale_factor:.1f} | Neighbors: {min_neighbors}"
        cv2.putText(
            frame,
            param_text,
            (10, 90),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (255, 255, 255),
            1
        )
        
        # Instructions
        cv2.putText(
            frame,
            "q: Quit | s: Screenshot | +/-: Sensitivity",
            (10, height - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (255, 255, 255),
            1
        )
        
        # Display frame
        cv2.imshow('Friend Classifier - Phase 2: Face Detection', frame)
        
        # Handle keyboard input
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q') or key == 27:  # Quit
            print("\n👋 Quitting...")
            break
        
        elif key == ord('s'):  # Screenshot
            screenshot_count += 1
            filename = os.path.join(screenshot_dir, f'face_detection_screenshot_{screenshot_count}.jpg')
            cv2.imwrite(filename, frame)
            print(f"📸 Screenshot saved: {filename}")
        
        elif key == ord('+') or key == ord('='):  # Increase sensitivity
            min_neighbors = max(1, min_neighbors - 1)
            print(f"🔧 Sensitivity increased (min_neighbors: {min_neighbors})")
        
        elif key == ord('-') or key == ord('_'):  # Decrease sensitivity
            min_neighbors = min(10, min_neighbors + 1)
            print(f"🔧 Sensitivity decreased (min_neighbors: {min_neighbors})")
    
    # Cleanup
    cap.release()
    cv2.destroyAllWindows()
    
    # Final statistics
    avg_faces = faces_detected_total / frame_count if frame_count > 0 else 0
    
    print("\n" + "="*60)
    print("✅ PHASE 2 COMPLETE!")
    print(f"   Total frames: {frame_count}")
    print(f"   Average FPS: {fps:.1f}")
    print(f"   Total faces detected: {faces_detected_total}")
    print(f"   Average faces per frame: {avg_faces:.2f}")
    print(f"   Screenshots saved: {screenshot_count}")
    print("="*60)

In [19]:

def test_face_detection_on_image(image_path):
    
    print(f"\n🖼️ Testing face detection on: {image_path}")
    
    # Load detector
    face_cascade = load_face_detector()
    if face_cascade is None:
        return
    
    # Load image
    frame = cv2.imread(image_path)
    if frame is None:
        print(f"❌ Could not load image: {image_path}")
        return
    
    # Detect faces
    faces = detect_faces(frame, face_cascade)
    
    print(f"✅ Detected {len(faces)} face(s)")
    
    # Draw boxes
    frame = draw_face_boxes(frame, faces)
    
    # Display
    cv2.imshow('Face Detection Test', frame)
    print("Press any key to close...")
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [20]:
def adjust_detection_parameters():
    """
    Interactive tool to tune face detection parameters.
    Helps you find the best settings for your environment.
    """
    
    print("\n🔧 DETECTION PARAMETER TUNER")
    print("="*60)
    print("This helps you find the best detection settings.")
    print("Adjust parameters and see results in real-time!")
    print("="*60)
    
    face_cascade = load_face_detector()
    if face_cascade is None:
        return
    
    cap = cv2.VideoCapture(1, cv2.CAP_AVFOUNDATION)
    if not cap.isOpened():
        print("❌ Failed to open camera")
        return
    
    # Parameters to tune
    scale_factor = 1.1
    min_neighbors = 5
    min_size = 30
    
    print("\nControls:")
    print("  1/2: Decrease/Increase scale_factor (detection speed vs accuracy)")
    print("  3/4: Decrease/Increase min_neighbors (fewer/more false positives)")
    print("  5/6: Decrease/Increase min_size (smaller/larger faces)")
    print("  q: Quit")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Detect with current parameters
        faces = detect_faces(
            frame, 
            face_cascade,
            scale_factor=scale_factor,
            min_neighbors=min_neighbors,
            min_size=(min_size, min_size)
        )
        
        # Draw results
        frame = draw_face_boxes(frame, faces)
        
        # Show parameters
        info_y = 30
        cv2.putText(frame, f"Faces: {len(faces)}", (10, info_y), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        info_y += 30
        cv2.putText(frame, f"scale_factor: {scale_factor:.2f} (1/2)", (10, info_y),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        info_y += 25
        cv2.putText(frame, f"min_neighbors: {min_neighbors} (3/4)", (10, info_y),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        info_y += 25
        cv2.putText(frame, f"min_size: {min_size}x{min_size} (5/6)", (10, info_y),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        cv2.imshow('Parameter Tuner', frame)
        
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q'):
            break
        elif key == ord('1'):
            scale_factor = max(1.05, scale_factor - 0.05)
            print(f"scale_factor: {scale_factor:.2f}")
        elif key == ord('2'):
            scale_factor = min(1.5, scale_factor + 0.05)
            print(f"scale_factor: {scale_factor:.2f}")
        elif key == ord('3'):
            min_neighbors = max(1, min_neighbors - 1)
            print(f"min_neighbors: {min_neighbors}")
        elif key == ord('4'):
            min_neighbors = min(10, min_neighbors + 1)
            print(f"min_neighbors: {min_neighbors}")
        elif key == ord('5'):
            min_size = max(10, min_size - 10)
            print(f"min_size: {min_size}x{min_size}")
        elif key == ord('6'):
            min_size = min(200, min_size + 10)
            print(f"min_size: {min_size}x{min_size}")
    
    cap.release()
    cv2.destroyAllWindows()
    
    print("\n✅ Final recommended settings:")
    print(f"   scale_factor = {scale_factor:.2f}")
    print(f"   min_neighbors = {min_neighbors}")
    print(f"   min_size = ({min_size}, {min_size})")


In [21]:

if __name__ == "__main__":
    # Option 1: Run face detection (main functionality)
    face_detection_camera(camera_index=1)
    
    # Uncomment to Tune parameters interactively
    adjust_detection_parameters()



🎥 PHASE 2: FACE DETECTION
📦 Loading Haar Cascade face detector...
✅ Face detector loaded successfully!

📹 Opening camera 1...
✅ Camera opened successfully!

📋 Controls:
   q - Quit
   s - Save screenshot
   + - Increase sensitivity (detect more faces)
   - - Decrease sensitivity (detect fewer faces)

Window opening...


👋 Quitting...

✅ PHASE 2 COMPLETE!
   Total frames: 88
   Average FPS: 7.4
   Total faces detected: 92
   Average faces per frame: 1.05
   Screenshots saved: 0

🔧 DETECTION PARAMETER TUNER
This helps you find the best detection settings.
Adjust parameters and see results in real-time!
📦 Loading Haar Cascade face detector...
✅ Face detector loaded successfully!

Controls:
  1/2: Decrease/Increase scale_factor (detection speed vs accuracy)
  3/4: Decrease/Increase min_neighbors (fewer/more false positives)
  5/6: Decrease/Increase min_size (smaller/larger faces)
  q: Quit
scale_factor: 1.05
scale_factor: 1.10
scale_factor: 1.15
scale_factor: 1.20
scale_factor: 1.25
scale